In [1]:
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv('train.csv')
df1= pd.read_csv('test.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [3]:
df.isna().count()


Loan_ID              614
Gender               614
Married              614
Dependents           614
Education            614
Self_Employed        614
ApplicantIncome      614
CoapplicantIncome    614
LoanAmount           614
Loan_Amount_Term     614
Credit_History       614
Property_Area        614
Loan_Status          614
dtype: int64

In [4]:
nonnum=df.select_dtypes(exclude=["number"]).columns

In [5]:
num = df._get_numeric_data().columns


In [6]:
df[nonnum]

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,Urban,Y
4,LP001008,Male,No,0,Graduate,No,Urban,Y
...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,Urban,Y


# PREPROCESSING PHASE

In [7]:
def preprocessing(df):
    col1=df['Dependents'].values
    for i in range(len(col1)):
        if col1[i]=='3+':
            col1[i]=3
    df['Dependents']= col1
    col1=df['Married'].values
    for i in range(len(col1)):
        if col1[i]=='Yes':
            col1[i]=1
        else:
            col1[i]=0
    df['Married']= col1
    col1=df['Education'].values
    for i in range(len(col1)):
        if col1[i]=='Graduate':
            col1[i]=1
        else:
            col1[i]=0
    df['Graduate']= col1
    col1=df['Self_Employed'].values
    for i in range(len(col1)):
        if col1[i]=='Yes':
            col1[i]=1
        else:
            col1[i]=0
    df['Self_Employed']= col1
    col1=df['Property_Area'].values
    for i in range(len(col1)):
        if col1[i]=='Urban':
            col1[i]=1
        elif col1[i]=='Semiurban':
            col1[i]=0.5
        else:
            col1[i]=0
    df['Property_Area']= col1
    
    col1=df['Gender'].values
    for i in range(len(col1)):
        if col1[i]=='Male':
            col1[i]=1
        else:
            col1[i]=-1
    df['Gender']= col1
    X= df[['Gender','Graduate','Married','Dependents','Education','ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area']].values
    
    from sklearn.impute import SimpleImputer
    imputer = SimpleImputer(strategy='mean')
    imputer.fit(X)
    X= imputer.transform(X)
    return X
    
    
    
    
    

In [8]:
col1=df['Loan_Status'].values
for i in range(len(col1)):
    if col1[i]=='Y':
        col1[i]=1
    else:
        col1[i]=0
df['Loan_Status']= col1
y= df['Loan_Status'].values

In [9]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
imputer.fit(y.reshape(-1,1))
y= imputer.transform(y.reshape(-1,1))

In [10]:
X_val= preprocessing(df1)
X= preprocessing(df)

WE CAN USE CLUSTERING AS WELL AS CLASSIFICATION

# CLASSIFICATION 

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,accuracy_score
clf= LogisticRegression()
clf.fit(X_train,y_train)
y_pred= clf.predict(X_test)
print('F1 Score',f1_score(y_test, y_pred))
print('Accuracy',accuracy_score(y_test,y_pred)*100)

C:\Users\HEMANTH\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


F1 Score 0.8426966292134831
Accuracy 77.23577235772358


C:\Users\HEMANTH\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [13]:
from xgboost import XGBClassifier
clf= XGBClassifier()
clf.fit(X_train,y_train)
y_pred= clf.predict(X_test)
print('F1 Score',f1_score(y_test, y_pred))
print('Accuracy',accuracy_score(y_test,y_pred)*100)

F1 Score 0.8181818181818182
Accuracy 73.98373983739837


C:\Users\HEMANTH\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\HEMANTH\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [14]:
from sklearn import svm
clf = svm.SVC(kernel="linear")
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print('F1 Score',f1_score(y_test, y_pred))
print('Accuracy',accuracy_score(y_test,y_pred)*100)

C:\Users\HEMANTH\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


F1 Score 0.8333333333333333
Accuracy 75.60975609756098


In [15]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf = PassiveAggressiveClassifier(max_iter=1000, random_state=0,tol=1e-3)
clf.fit(X_train, y_train)
y_pred= clf.predict(X_test)
print('F1 Score',f1_score(y_test, y_pred))
print('Accuracy',accuracy_score(y_test,y_pred)*100)


F1 Score 0.6666666666666667
Accuracy 51.21951219512195


C:\Users\HEMANTH\anaconda3\lib\site-packages\sklearn\utils\validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print('F1 Score',f1_score(y_test, y_pred))
print('Accuracy',accuracy_score(y_test,y_pred)*100)

F1 Score 0.8295454545454546
Accuracy 75.60975609756098


C:\Users\HEMANTH\anaconda3\lib\site-packages\sklearn\naive_bayes.py:206: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print('F1 Score',f1_score(y_test, y_pred))
print('Accuracy',accuracy_score(y_test,y_pred)*100)

C:\Users\HEMANTH\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


F1 Score 0.8508287292817679
Accuracy 78.04878048780488


In [18]:
from sklearn.neighbors import KNeighborsClassifier
knn1 = KNeighborsClassifier(n_neighbors=3)
knn1.fit(X_train, y_train)
y_pred = knn1.predict(X_test)
print("Score is",knn1.score(X_test,y_test)*100)

Score is 52.84552845528455


C:\Users\HEMANTH\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
def create_baseline():
	# create model
	model = Sequential()
	model.add(Dense(60, input_dim=11, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [ ]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [ ]:
#Best one was RandomForest Classifier